# DL

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [9]:
from fastai.structured import *
from fastai.column_data import *
import pandas as pd
import keras
import utils
from keras.models import model_from_yaml
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 20
import numpy as np
from pandas_summary import DataFrameSummary
from importlib import reload
reload(utils)

np.set_printoptions(threshold=50, edgeitems=20)

PATH='./data/'

## Create Data Set

In [4]:
# load data
!ls ./data

2model_loss.csv
air_day_reservation.csv
air_reserve.csv
air_store_info.csv
air_store_info_with_nearest_active_station.csv
air_visit_data.csv
air_visit_data_with_prop.csv
air_visit_data_with_prop_feas.csv
air_visit_data_with_prop_feas_2.csv
air_visit_data_with_weather.csv
area_date_feas.csv
area_dow_feas.csv
date_info.csv
feas.csv
hpg_reserve.csv
hpg_store_info.csv
model_selection.csv
pp_cv_loss_base_model.csv
pp_cv_loss_base_model_for_test.csv
sample_submission.csv
store_id_relation.csv
submission_data_with_prop.csv
test_data_with_weather.csv
weather
weather_data_merge.csv
weather_stations.csv
xbg_prob_loss.csv
xgb_beat_prop_stores.csv


In [24]:
trn = pd.read_csv('{}/air_visit_data.csv'.format('./data'))
train_feas = utils.data2fea(trn, './data', drop_vars=[])

In [25]:
DataFrameSummary(train_feas['tidy_data']).summary()

,air_store_id,visit_date,visitors,genre_name,area_name,hpb_genre_name,hpb_area_name,air_loc,hpb_loc,stores_in_air_loc,...,mean_visits_in_hpb_loc,std_visits_in_hpb_loc,min_visits_in_area_name,max_visits_in_area_name,mean_visits_in_area_name,std_visits_in_area_name,min_visits_in_hpb_area_name,max_visits_in_hpb_area_name,mean_visits_in_hpb_area_name,std_visits_in_hpb_area_name
count,NaN,NaN,252108,NaN,NaN,NaN,NaN,NaN,NaN,97333,...,97333,97333,97333,97333,97333,97333,18413,18413,18413,18413
mean,NaN,NaN,20.9738,NaN,NaN,NaN,NaN,NaN,NaN,9.42647,...,22.3639,16.6559,1.07246,141.107,22.3639,15.8231,1.18172,115.027,24.7742,16.2426
std,NaN,NaN,16.757,NaN,NaN,NaN,NaN,NaN,NaN,6.44106,...,3.15909,2.21379,0.584741,61.0932,5.24641,3.80518,0.529489,56.4857,6.75216,5.06922
min,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,...,11.5012,8.03152,1,27,8.43088,5.21373,1,41,11.5012,8.03152
25%,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,3,...,21.8016,16.7523,1,90,18.8616,14.1494,1,79,18.2677,12.1287
50%,NaN,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,10,...,21.8016,16.7523,1,126,22.9268,16.1833,1,101,24.9981,14.3866
75%,NaN,NaN,29,NaN,NaN,NaN,NaN,NaN,NaN,15,...,21.8016,16.7523,1,199,25.2504,18.8833,1,128,29.1217,20.7986
max,NaN,NaN,877,NaN,NaN,NaN,NaN,NaN,NaN,22,...,41.4439,26.6851,8,262,42.4602,23.477,4,244,41.4439,26.6851
counts,252108,252108,252108,97333,97333,18413,18413,97333,97333,97333,...,97333,97333,97333,97333,97333,97333,18413,18413,18413,18413
uniques,829,478,204,14,71,15,32,73,33,14,...,33,33,6,56,71,71,3,28,32,32


In [13]:
visit_data_pp = pd.read_csv(f'{PATH}air_visit_data_with_prop_feas_2.csv')
store_feas = pd.read_csv(f'{PATH}feas.csv')
weather_feas = pd.read_csv(f'{PATH}weather_data_merge.csv')

In [15]:
display(visit_data_pp.head())
display(visit_data_pp.columns)
display(DataFrameSummary(visit_data_pp).summary())

,Unnamed: 0,air_store_id,visit_date,visitors,yhat,yhat_upper,yhat_lower,trend,trend_upper,trend_lower,...,holidays_lower,seasonal,seasonal_upper,seasonal_lower,seasonalities,seasonalities_upper,seasonalities_lower,weekly,weekly_upper,weekly_lower
0,0,air_ba937bf13d40fb24,13/1/2016,25,20.826006,37.535546,11.941791,14.452958,14.452958,14.452958,...,1.0,1.440951,1.440951,1.440951,1.440951,1.440951,1.440951,1.440951,1.440951,1.440951
1,1,air_ba937bf13d40fb24,14/1/2016,32,17.580470,29.262021,10.042691,14.478052,14.478052,14.478052,...,1.0,1.214284,1.214284,1.214284,1.214284,1.214284,1.214284,1.214284,1.214284,1.214284
2,2,air_ba937bf13d40fb24,15/1/2016,29,30.992940,54.328539,17.723215,14.503190,14.503190,14.503190,...,1.0,2.136974,2.136974,2.136974,2.136974,2.136974,2.136974,2.136974,2.136974,2.136974
3,3,air_ba937bf13d40fb24,16/1/2016,22,24.052839,42.666218,13.724146,14.528371,14.528371,14.528371,...,1.0,1.655577,1.655577,1.655577,1.655577,1.655577,1.655577,1.655577,1.655577,1.655577
4,4,air_ba937bf13d40fb24,18/1/2016,6,10.986390,19.049318,6.315840,14.578864,14.578864,14.578864,...,1.0,0.753583,0.753583,0.753583,0.753583,0.753583,0.753583,0.753583,0.753583,0.753583


Index(['Unnamed: 0', 'air_store_id', 'visit_date', 'visitors', 'yhat',
       'yhat_upper', 'yhat_lower', 'trend', 'trend_upper', 'trend_lower',
       'holidays', 'holidays_upper', 'holidays_lower', 'seasonal',
       'seasonal_upper', 'seasonal_lower', 'seasonalities',
       'seasonalities_upper', 'seasonalities_lower', 'weekly', 'weekly_upper',
       'weekly_lower'],
      dtype='object')

,Unnamed: 0,air_store_id,visit_date,visitors,yhat,yhat_upper,yhat_lower,trend,trend_upper,trend_lower,...,holidays_lower,seasonal,seasonal_upper,seasonal_lower,seasonalities,seasonalities_upper,seasonalities_lower,weekly,weekly_upper,weekly_lower
count,252108,NaN,NaN,252108,252108,252108,252108,252108,252108,252108,...,252108,252108,252108,252108,252108,252108,252108,252108,252108,252108
mean,126054,NaN,NaN,20.9738,18.872,36.2924,10.2297,16.7066,16.7066,16.7066,...,1.0164,1.14264,1.14264,1.14264,1.12732,1.12732,1.12732,1.12732,1.12732,1.12732
std,72777.5,NaN,NaN,16.757,12.8735,23.7684,8.18572,9.71724,9.71724,9.71724,...,0.156262,0.439532,0.439532,0.439532,0.404204,0.404204,0.404204,0.404204,0.404204,0.404204
min,0,NaN,NaN,1,0.503156,1.0741,0.201433,0.816253,0.816253,0.816253,...,0.0286583,0.0257218,0.0257218,0.0257218,0.0460737,0.0460737,0.0460737,0.0460737,0.0460737,0.0460737
25%,63026.8,NaN,NaN,9,9.03357,19.2976,4.14959,8.95559,8.95559,8.95559,...,1,0.863207,0.863207,0.863207,0.860852,0.860852,0.860852,0.860852,0.860852,0.860852
50%,126054,NaN,NaN,17,15.8303,31.1566,7.85494,14.5345,14.5345,14.5345,...,1,1.05448,1.05448,1.05448,1.0439,1.0439,1.0439,1.0439,1.0439,1.0439
75%,189080,NaN,NaN,29,25.5152,47.5329,14.1567,23.0372,23.0372,23.0372,...,1,1.34784,1.34784,1.34784,1.33346,1.33346,1.33346,1.33346,1.33346,1.33346
max,252107,NaN,NaN,877,307.498,715.356,170.832,131.658,131.658,131.658,...,7.68166,15.5166,15.5166,15.5166,8.00954,8.00954,8.00954,8.00954,8.00954,8.00954
counts,252108,252108,252108,252108,252108,252108,252108,252108,252108,252108,...,252108,252108,252108,252108,252108,252108,252108,252108,252108,252108
uniques,252108,829,478,204,252101,252107,252105,252101,252101,252101,...,2502,14356,14356,14356,5753,5753,5753,5753,5753,5753


In [16]:
display(store_feas.head())
display(store_feas.columns)
display(DataFrameSummary(store_feas).summary())

,air_store_id,visit_date,visitors,genre_name,area_name,hpb_genre_name,hpb_area_name,air_loc,hpb_loc,stores_in_air_loc,...,mean_visits_in_hpb_loc,std_visits_in_hpb_loc,min_visits_in_area_name,max_visits_in_area_name,mean_visits_in_area_name,std_visits_in_area_name,min_visits_in_hpb_area_name,max_visits_in_hpb_area_name,mean_visits_in_hpb_area_name,std_visits_in_hpb_area_name
0,air_00a91d42b08b08d9,2016-07-01,35,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,NaN,NaN,35.6940027_139.7535951,nan_nan,10.0,...,21.801584,16.752281,1.0,130.0,30.483187,17.644729,NaN,NaN,NaN,NaN
1,air_00a91d42b08b08d9,2016-07-02,9,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,NaN,NaN,35.6940027_139.7535951,nan_nan,10.0,...,21.801584,16.752281,1.0,130.0,30.483187,17.644729,NaN,NaN,NaN,NaN
2,air_00a91d42b08b08d9,2016-07-04,20,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,NaN,NaN,35.6940027_139.7535951,nan_nan,10.0,...,21.801584,16.752281,1.0,130.0,30.483187,17.644729,NaN,NaN,NaN,NaN
3,air_00a91d42b08b08d9,2016-07-05,25,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,NaN,NaN,35.6940027_139.7535951,nan_nan,10.0,...,21.801584,16.752281,1.0,130.0,30.483187,17.644729,NaN,NaN,NaN,NaN
4,air_00a91d42b08b08d9,2016-07-06,29,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,NaN,NaN,35.6940027_139.7535951,nan_nan,10.0,...,21.801584,16.752281,1.0,130.0,30.483187,17.644729,NaN,NaN,NaN,NaN


Index(['air_store_id', 'visit_date', 'visitors', 'genre_name', 'area_name',
       'hpb_genre_name', 'hpb_area_name', 'air_loc', 'hpb_loc',
       'stores_in_air_loc', 'stores_in_hpb_loc', 'stores_in_area_name',
       'stores_in_hpb_area_name', 'min_visits_in_air_store_id',
       'max_visits_in_air_store_id', 'mean_visits_in_air_store_id',
       'std_visits_in_air_store_id', 'min_visits_in_air_loc',
       'max_visits_in_air_loc', 'mean_visits_in_air_loc',
       'std_visits_in_air_loc', 'min_visits_in_hpb_loc',
       'max_visits_in_hpb_loc', 'mean_visits_in_hpb_loc',
       'std_visits_in_hpb_loc', 'min_visits_in_area_name',
       'max_visits_in_area_name', 'mean_visits_in_area_name',
       'std_visits_in_area_name', 'min_visits_in_hpb_area_name',
       'max_visits_in_hpb_area_name', 'mean_visits_in_hpb_area_name',
       'std_visits_in_hpb_area_name'],
      dtype='object')

,air_store_id,visit_date,visitors,genre_name,area_name,hpb_genre_name,hpb_area_name,air_loc,hpb_loc,stores_in_air_loc,...,mean_visits_in_hpb_loc,std_visits_in_hpb_loc,min_visits_in_area_name,max_visits_in_area_name,mean_visits_in_area_name,std_visits_in_area_name,min_visits_in_hpb_area_name,max_visits_in_hpb_area_name,mean_visits_in_hpb_area_name,std_visits_in_hpb_area_name
count,NaN,NaN,250468,NaN,NaN,NaN,NaN,NaN,NaN,97333,...,97333,97333,97333,97333,97333,97333,18413,18413,18413,18413
mean,NaN,NaN,20.9593,NaN,NaN,NaN,NaN,NaN,NaN,9.42647,...,22.3639,16.6559,1.07246,141.107,22.3639,15.8231,1.18172,115.027,24.7742,16.2426
std,NaN,NaN,16.7398,NaN,NaN,NaN,NaN,NaN,NaN,6.44106,...,3.15909,2.21379,0.584741,61.0932,5.24641,3.80518,0.529489,56.4857,6.75216,5.06922
min,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,...,11.5012,8.03152,1,27,8.43088,5.21373,1,41,11.5012,8.03152
25%,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,3,...,21.8016,16.7523,1,90,18.8616,14.1494,1,79,18.2677,12.1287
50%,NaN,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,10,...,21.8016,16.7523,1,126,22.9268,16.1833,1,101,24.9981,14.3866
75%,NaN,NaN,29,NaN,NaN,NaN,NaN,NaN,NaN,15,...,21.8016,16.7523,1,199,25.2504,18.8833,1,128,29.1217,20.7986
max,NaN,NaN,877,NaN,NaN,NaN,NaN,NaN,NaN,22,...,41.4439,26.6851,8,262,42.4602,23.477,4,244,41.4439,26.6851
counts,250468,250468,250468,97333,97333,18413,18413,97333,97333,97333,...,97333,97333,97333,97333,97333,97333,18413,18413,18413,18413
uniques,821,478,203,14,71,15,32,73,33,14,...,33,33,6,56,71,71,3,28,32,32


In [17]:
display(weather_feas.head())
display(weather_feas.columns)
display(DataFrameSummary(weather_feas).summary())

,visit_date,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,...,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_latitude,station_longitude,station_vincenty,station_great_circle
0,2016-01-01,7.9,12.3,4.8,0.0,8.6,NaN,NaN,NaN,2.8,...,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",34.696667,135.211667,1.277232,1.274882
1,2016-01-01,7.9,12.3,4.8,0.0,8.6,NaN,NaN,NaN,2.8,...,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",34.696667,135.211667,1.277232,1.274882
2,2016-01-01,7.9,12.3,4.8,0.0,8.6,NaN,NaN,NaN,2.8,...,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",34.696667,135.211667,1.277232,1.274882
3,2016-01-01,7.9,12.3,4.8,0.0,8.6,NaN,NaN,NaN,2.8,...,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",34.696667,135.211667,1.277232,1.274882
4,2016-01-01,7.9,12.3,4.8,0.0,8.6,NaN,NaN,NaN,2.8,...,Dining bar,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,"""34.6951242""","""135.1978525""",34.696667,135.211667,1.277232,1.274882


Index(['visit_date', 'avg_temperature', 'high_temperature', 'low_temperature',
       'precipitation', 'hours_sunlight', 'solar_radiation',
       'deepest_snowfall', 'total_snowfall', 'avg_wind_speed',
       'avg_vapor_pressure', 'avg_local_pressure', 'avg_humidity',
       'avg_sea_pressure', 'cloud_cover', 'station_id', 'air_store_id',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'latitude_str', 'longitude_str', 'station_latitude',
       'station_longitude', 'station_vincenty', 'station_great_circle'],
      dtype='object')

,visit_date,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,...,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_latitude,station_longitude,station_vincenty,station_great_circle
count,NaN,385165,385161,385161,294513,366948,287452,16418,9183,384972,...,NaN,NaN,428593,428593,NaN,NaN,428593,428593,428593,428593
mean,NaN,14.6977,19.144,10.8993,5.91825,5.52442,13.5146,33.2732,4.7197,2.85767,...,NaN,NaN,35.647,137.415,NaN,NaN,35.6484,137.415,3.83125,3.83143
std,NaN,8.2191,8.46122,8.50146,14.052,4.10359,7.41456,29.4018,6.00985,1.14048,...,NaN,NaN,2.0834,3.64877,NaN,NaN,2.08328,3.64843,2.21526,2.21517
min,NaN,-18.8,-10.6,-29.1,0,0,0.48,0,0,0.3,...,NaN,NaN,33.212,130.196,NaN,NaN,33.225,130.19,0.416011,0.415906
25%,NaN,7.8,12.3,3.7,0,1.3,7.55,2,1,2.1,...,NaN,NaN,34.6951,135.342,NaN,NaN,34.6817,135.335,1.939,1.93624
50%,NaN,14.5,19.2,10.2,0,5.7,12.26,32,3,2.7,...,NaN,NaN,35.6581,139.685,NaN,NaN,35.6917,139.62,3.57524,3.57845
75%,NaN,21.6,26.1,18,5,9,19.13,58,6,3.4,...,NaN,NaN,35.694,139.752,NaN,NaN,35.6917,139.75,5.31996,5.31864
max,NaN,32.4,38.3,28.9,208,14.8,31.32,96,69,12.8,...,NaN,NaN,44.0206,144.273,NaN,NaN,44.0167,144.278,12.2173,12.19
counts,428593,385165,385161,385161,294513,366948,287452,16418,9183,384972,...,428593,428593,428593,428593,428593,428593,428593,428593,428593,428593
uniques,517,452,463,505,214,147,2402,94,35,110,...,14,103,108,108,108,108,38,38,108,108


In [20]:
test_raw = pd.read_csv(f'{PATH}sample_submission.csv')
test_data = utils.tes2trn(test_raw)
test_stores = test_data.air_store_id.unique()
len(test_stores)

821

remove stores which are not in test set

In [21]:
joined = visit_data_pp[visit_data_pp.air_store_id.isin(test_stores)]

In [22]:
def dropMissingColumns

821